# Analysis of the covid moonshot rationale
`COVID_moonshot_submissions` is the gitrepo https://github.com/postera-ai/COVID_moonshot_submissions

In [11]:
import os
user = 'postera-ai'
repo_name = 'COVID_moonshot_submissions'
filename = 'downloaded_COVID_submissions_file.csv'

from github import Github
repo = Github().get_user(user).get_repo(repo_name)
spreadsheet_url = repo.get_contents(filename).download_url

# ----------------------------------------------------------------------------------------------------

import requests

response: requests.Response = requests.get(spreadsheet_url)
response.raise_for_status()

# ----------------------------------------------------------------------------------------------------

import pandas as pd
from io import StringIO

moonshot = pd.read_csv(StringIO(response.text), low_memory=False)
# the table contains multiple rows/submitted compounds per submission.
# Say the CID `ANT-DIA-3c79be55-1` finishes in `-1`

_choices = {row.name: set(map(type, row.unique())) for i, row in moonshot.transpose().iterrows()}
moonshot = moonshot.assign(CID_group=moonshot.CID.str.extract(r'(.*)\-\d+'),
                            **{k: moonshot[k].fillna(False).astype(bool) for k, v in _choices.items() if bool in v},
                            **{k: moonshot[k].fillna('').astype(str) for k, v in _choices.items() if str in v})

In [117]:
moonshot.to_pickle('moonshot_submissions.p')

## Bag of words

Making a bag of words for a word cloud and to see which terms are enriched.
However, common words need to be removed.
But not common words with a special meaning.
Unfortunately I did not find a tool that fixed grammatical inflections, hence the simple rule applied.

In [62]:
import re
from collections import Counter
moonshot['words'] = (moonshot.rationale + ' ' + moonshot['Submission Notes'])\
                     .str.replace('non ', 'non-')\
                     .str.replace('by eye', 'by-eye')\
                     .str.replace("n't", '')\
                     .str.lower()
minimoonshot = moonshot.drop_duplicates('CID_group')
word_block = ' '.join(minimoonshot.words)
words = re.findall('[\w\-]+', word_block)
wordbag = Counter(words)

In [63]:
wordbag['fragalysis']

41

In [64]:
# common words (`the`, `of` etc.) need to be remove from the wordbag

import requests

r = requests.get('https://gist.githubusercontent.com/deekayen/4148741/raw/98d35708fa344717d8eee15d11987de6c8e26d7d/1-1000.txt')

common_words = set(r.text.split('\n')) - {'molecule', 'machine', 'dock', 'learn'}

# shoddy way of expanding it.
# `againsted` is not a real word, but its presence does not harm anyone.
expanded_common_words = set(list(common_words) + \
                            [w+'s' for w in common_words] + \
                            [w+'es' for w in common_words] + \
                            [w+'ed' for w in common_words] + \
                            [w+'ing' for w in common_words] + \
                            [w+'ly' for w in common_words] + \
                            ['http', 'https', 'com', 'org', 'www']
                           )

In [65]:
## remove them

for k in expanded_common_words.intersection(wordbag.keys()):
    del wordbag[k]
    
for k in list(wordbag.keys()):
    if k.isdigit() or len(k) < 3:
        del wordbag[k]

In [66]:
def get_inflection(word:str):
    for inflection in ('ings', 's', 'ed', 'ing', 'ly'):
        if word[-len(inflection):] == inflection:
            return inflection
    else:
        return None
            
for word in list(wordbag.keys()):
    inflection = get_inflection(word)
    if inflection is None:
        continue
    short = word[:len(word)-len(inflection)]
    if short in wordbag:
        pass
    elif len(short) < 3:
        continue
    elif short[-1] == 'e' and short[:-1] in wordbag:
        short = short[-1]
    elif short+'e' in wordbag:
        short = short+'e'
    elif short[-2:] == 'ie' and short[-2:]+'y' in wordbag:
        short = short[-2:]+'y'
    elif word[-3:] == 'ses' and word[:-3]+'sis' in wordbag:
        short = word[:-3]+'sis'
    else:
        continue
    wordbag[short] += wordbag[word]
    del wordbag[word]

In [67]:
wordbag['hit'] += wordbag['hts']
del wordbag['hts']

In [87]:
for k in set(['file', 'assume', 'given', 'built', 'remove', 'previous', 'extra', 'mode', 'via', 'initial', 'report']).intersection(wordbag.keys()):
    del wordbag[k]

In [81]:
wordbag['submission'] += wordbag['submitted']
del wordbag['submitted']

In [89]:
wordbag.most_common(10)

[('fragment', 865),
 ('compound', 815),
 ('structure', 746),
 ('molecule', 646),
 ('dock', 600),
 ('bind', 570),
 ('submission', 486),
 ('bond', 435),
 ('ligand', 426),
 ('chain', 344)]

## Extra removal

Some XChem IDs and SMILES made it into the the list.

In [104]:
del wordbag['nan']

for word in list(wordbag.keys()):
    # if set(word).intersection('aeiouy') == set():
    #     print(word)
    if set(word).difference('conxpjs01234567890_') == set():
        del wordbag[word]

## Save

File made for https://www.wordclouds.co.uk/

In [88]:
# make word cloud
with open('wordcloud.txt', 'w') as w:
    w.write('\n'.join([' '.join([k] * wordbag[k]) for k in wordbag if wordbag[k] > 20 and len(k) > 2]))

## Find enrichment

In [105]:
from typing import Dict

def get_data_on_term(term:str) -> Dict[str, float]:
    print(term)
    subtable = moonshot.loc[moonshot.words.str.contains(term, case=False)]
    return {'term': term, **get_data_on_table(subtable)}
    
def get_data_on_table(table:pd.DataFrame) -> Dict[str, float]:
    data = {}
    data['N'] = len(table)
    if data['N'] == 0:
        print(f'Table failed!')
        return data
    # ordered
    ordered = table.ORDERED.value_counts()
    data['N_ordered'] = ordered[True] if True in ordered else 0
    data['N_not_ordered'] = ordered[False] if False in ordered else 0
    data['freq_ordered (of total)'] = round(data['N_ordered'] / data['N'], 4)
    # made
    made = table.MADE.value_counts()
    data['N_made'] = made[True] if True in made else 0
    data['N_not_made'] = made[False] if False in made else 0
    data['freq_made (of total)'] = round(data['N_made'] / data['N'], 4)
    data['freq_made (of ordered)'] = round(data['N_made']/data['N_ordered'], 4) if data['N_ordered'] != 0 else 0
    # xstalised
    xstalised = table['Structure ID'].value_counts().sum()
    data['N_crystallised'] = xstalised
    data['N_not_crystallised'] = data['N'] - xstalised
    data['freq_crystallised (of total)'] = round(xstalised/data['N'], 4)
    data['freq_crystallised (of made)'] = round(xstalised/data['N_made'], 4) if data['N_made'] != 0 else 0
    # assayed
    assayed = table.ASSAYED.value_counts()
    data['N_assayed'] = assayed[True] if True in assayed else 0
    data['N_not_assayed'] = assayed[False] if False in assayed else 0
    data['freq_assayed (of total)'] = round(data['N_assayed'] / data['N'], 4)
    data['freq_assayed (of made)'] = round(data['N_assayed']/data['N_made'], 4)  if data['N_made'] != 0 else 0
    # return
    return data

In [106]:
# pd.DataFrame({'term': pd.Series(data = list(wordbag.keys()), dtype=str),
#               'count': pd.Series(data = list(wordbag.values()), dtype=int)})

In [ ]:
data = [get_data_on_term(term) for term in wordbag]

In [109]:
term_table.to_csv(f'term_frequencies.csv')

In [111]:
data.append({'term': '<TALLY>', **get_data_on_table(moonshot)})
term_table = pd.DataFrame(data)

In [115]:
tally_row = term_table.loc[term_table.term == '<TALLY>'].iloc[0]

term_table = term_table[['term', 'N', 
                         'N_ordered', 'N_not_ordered',
                         'N_made', 'N_not_made',
                         'N_assayed', 'N_not_assayed',
                         'N_crystallised', 'N_not_crystallised',
                         'freq_ordered (of total)',
                         'freq_made (of total)', 
                         'freq_assayed (of total)',
                         'freq_crystallised (of total)',
                         'freq_made (of ordered)', 
                         'freq_assayed (of made)', 
                         'freq_crystallised (of made)'
                        ]]

term_table

,term,N,N_ordered,N_not_ordered,N_made,N_not_made,N_assayed,N_not_assayed,N_crystallised,N_not_crystallised,freq_ordered (of total),freq_made (of total),freq_assayed (of total),freq_crystallised (of total),freq_made (of ordered),freq_assayed (of made),freq_crystallised (of made)
0,nitrile,730,33,697,24,706,12,718,730,0,0.0452,0.0329,0.0164,1.0,0.7273,0.5000,30.4167
1,superimpose,56,13,43,5,51,4,52,56,0,0.2321,0.0893,0.0714,1.0,0.3846,0.8000,11.2000
2,ethylamine,12,1,11,1,11,0,12,12,0,0.0833,0.0833,0.0000,1.0,1.0000,0.0000,12.0000
3,structure,5993,358,5635,268,5725,154,5839,5993,0,0.0597,0.0447,0.0257,1.0,0.7486,0.5746,22.3619
4,maybe,92,10,82,8,84,4,88,92,0,0.1087,0.0870,0.0435,1.0,0.8000,0.5000,11.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6248,stuff,2,0,2,0,2,0,2,2,0,0.0000,0.0000,0.0000,1.0,0.0000,0.0000,0.0000
6249,en300-26666824,1,0,1,0,1,0,1,1,0,0.0000,0.0000,0.0000,1.0,0.0000,0.0000,0.0000
6250,e,19507,3677,15830,3058,16449,2055,17452,19507,0,0.1885,0.1568,0.1053,1.0,0.8317,0.6720,6.3790
6251,hit,4982,635,4347,575,4407,296,4686,4982,0,0.1275,0.1154,0.0594,1.0,0.9055,0.5148,8.6643


In [121]:
field = 'crystallised'
def chiify(row):
    chisquare(f_obs=[row.N, row[f'N_crystallised']],
                                   f_exp=[tally_row.N, tally_row[f'N_{field}'] ]).pvalue

,term,N,N_ordered,N_not_ordered,N_made,N_not_made,N_assayed,N_not_assayed,N_crystallised,N_not_crystallised,...,freq_made (of ordered),freq_assayed (of made),freq_crystallised (of made),log2_freq_ordered (of total),log2_freq_made (of total),log2_freq_assayed (of total),log2_freq_crystallised (of total),log2_freq_made (of ordered),log2_freq_assayed (of made),log2_freq_crystallised (of made)
0,nitrile,730,33,697,24,706,12,718,730,0,...,0.7273,0.5000,30.4167,-2.182636,-2.336761,-2.837205,0.0,-0.155549,-0.495491,2.337915
1,superimpose,56,13,43,5,51,4,52,56,0,...,0.3846,0.8000,11.2000,0.177716,-0.896188,-0.714977,0.0,-1.074741,0.182581,0.896550
2,ethylamine,12,1,11,1,11,0,12,12,0,...,1.0000,0.0000,12.0000,-1.300642,-0.996532,-inf,0.0,0.303828,-inf,0.996086
3,structure,5993,358,5635,268,5725,154,5839,5993,0,...,0.7486,0.5746,22.3619,-1.781228,-1.894574,-2.189132,0.0,-0.113905,-0.294861,1.894094
4,maybe,92,10,82,8,84,4,88,92,0,...,0.8000,0.5000,11.5000,-0.916679,-0.933833,-1.429885,0.0,-0.018100,-0.495491,0.934685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6248,stuff,2,0,2,0,2,0,2,2,0,...,0.0000,0.0000,0.0000,-inf,-inf,-inf,0.0,-inf,-inf,-inf
6249,en300-26666824,1,0,1,0,1,0,1,1,0,...,0.0000,0.0000,0.0000,-inf,-inf,-inf,0.0,-inf,-inf,-inf
6250,e,19507,3677,15830,3058,16449,2055,17452,19507,0,...,0.8317,0.6720,6.3790,-0.122466,-0.083995,-0.154467,0.0,0.037963,-0.068957,0.084454
6251,hit,4982,635,4347,575,4407,296,4686,4982,0,...,0.9055,0.5148,8.6643,-0.686533,-0.526277,-0.980438,0.0,0.160615,-0.453407,0.526207


In [125]:
from scipy.stats import chisquare, nbinom, fisher_exact


        

new_columns = {}
for field in ('ordered', 'made',  'assayed', 'crystallised'):
    fisherise = lambda row: fisher_exact([[row.N, row[f'N_{field}'] ],
                                          [tally_row.N, tally_row[f'N_{field}'] ]
                                         ])[1] if str(row[f'N_{field}']) != 'nan' else float('nan')
    #chiify = lambda row:  if row.N * row[f'N_{field}'] != 0 and str(row[f'N_{field}']) != 'nan' else float('nan')
    def chiify(row):
        try:
            return chisquare(f_obs=[row.N, row[f'N_{field}']],
                                       f_exp=[tally_row.N, tally_row[f'N_{field}'] ]).pvalue
        except Exception:
            return float('nan')
    new_columns[f'fisher_p_{field}'] = term_table.apply(fisherise,axis=1)
    new_columns[f'χ2_p_{field}'] = term_table.apply(chiify,axis=1)

term_table = term_table.assign(**new_columns)

There is something wrong with the Fischer test

In [135]:
import numpy as np

new_columns = {}
for field in ('freq_ordered (of total)',
              'freq_made (of total)',
              'freq_assayed (of total)', 
              'freq_crystallised (of total)',
              'freq_made (of ordered)', 
              'freq_assayed (of made)',
              'freq_crystallised (of made)'):
    new_columns[f'fold_{field}'] = term_table[field].values / tally_row[field]
    new_columns[f'log2_{field}'] = np.log2(new_columns[f'fold_{field}']) 

term_table = term_table.assign(**new_columns)

/var/folders/1q/8kyncjb50d512vj7d5tmx36h0000gn/T/ipykernel_47681/1363341477.py:12: RuntimeWarning: divide by zero encountered in log2
  new_columns[f'log2_{field}'] = np.log2(term_table[field].values / tally_row[field])


In [ ]:
term_table.to_csv(f'terms.csv')

In [130]:
## Crystallised

total = len(term_table)
crystal = term_table.loc[term_table.fisher_p_crystallised < 0.05/total]\
                    .sort_values('freq_crystallised (of total)', ascending=False)\
                    .head(100)
crystal.to_csv(f'Top100_terms_enchriched_for_crystallisation.csv')
crystal

,term,N,N_ordered,N_not_ordered,N_made,N_not_made,N_assayed,N_not_assayed,N_crystallised,N_not_crystallised,...,log2_freq_assayed (of made),log2_freq_crystallised (of made),fisher_p_ordered,χ2_p_ordered,fisher_p_made,χ2_p_made,fisher_p_assayed,χ2_p_assayed,fisher_p_crystallised,χ2_p_crystallised


In [133]:
## Crystallised 2

total = len(term_table)
crystal = term_table.loc[term_table.fisher_p_crystallised < 0.05/total]\
                    .sort_values('freq_crystallised (of made)', ascending=False)\
                    .head(100)
crystal.to_csv(f'Top100_terms_enchriched_for_crystallisation_of_made.csv')
crystal

,term,N,N_ordered,N_not_ordered,N_made,N_not_made,N_assayed,N_not_assayed,N_crystallised,N_not_crystallised,...,log2_freq_assayed (of made),log2_freq_crystallised (of made),fisher_p_ordered,χ2_p_ordered,fisher_p_made,χ2_p_made,fisher_p_assayed,χ2_p_assayed,fisher_p_crystallised,χ2_p_crystallised


In [134]:
headers = ['term','N','N_ordered','N_made','N_assayed','N_crystallised','fisher_p_made','fisher_p_crystallised','log2_freq_made (of total)','log2_freq_crystallised (of total)','log2_freq_crystallised (of made)']
term_table.loc[term_table.term.isin(['dock', 'score', 'merge', 'enumerate', 'calculate', 'by eye', 'fragalysis'])][headers]#.to_markdown()

,term,N,N_ordered,N_made,N_assayed,N_crystallised,fisher_p_made,fisher_p_crystallised,log2_freq_made (of total),log2_freq_crystallised (of total),log2_freq_crystallised (of made)
5,merge,395,59,27,12,395,6.564989e-07,1.0,-1.280852,0.0,1.281942
8,fragalysis,423,39,36,10,423,3.464785e-05,1.0,-0.965689,0.0,0.965712
1378,dock,5959,326,235,109,5959,2.600007e-139,1.0,-2.076653,0.0,2.075458
2628,enumerate,127,25,19,17,127,8.119545e-01,1.0,-0.151810,0.0,0.151878
